In [ ]:
from tune_util import *
import numpy as np

In [ ]:
import cma
import random
import configparser
import os
episodes_config = configparser.ConfigParser(
    allow_no_value=True, inline_comment_prefixes=";"
)
episodes_config.read(os.path.join(os.getcwd(), "params/episode_params_val.ini"))
series = episodes_config.sections()[1:]

In [ ]:
def_num = np.array([10,1,3,1000,4,1,0.8,2,0.35,2,3])
def_num_log = np.array(list(np.log(def_num)) + [0.0])
train_type = 'global' #'global', 'batch', 'individual'
port_num = 2112
suffix = '_long'
seq = 't_univ_across' #'t_zara1_dense_crosswrap_rev' 't_eth_dense_against' 't_univ_across' 't_dhotel_bottopmid'

In [ ]:
my_log = []
# res = exec_seqs(def_num_log,'test_eval',[random.choice(series)],log_file=my_log)
# res

In [ ]:
es = cma.CMAEvolutionStrategy(def_num_log, 0.1)

batch_iter = 0
max_batch = 100 #len(series)*15 if train_type == 'batch' else 15

seq_idxs = list(range(len(series)))

while batch_iter < max_batch:
    if((train_type == 'batch') and (batch_iter % len(series) == 0)):
        random.shuffle(seq_idxs)
    solutions = es.ask()
    r_vals = []
    if(train_type == 'global'):
        for params in solutions:
            res = exec_seqs(params, 'cma_' + train_type, series, log_file=my_log, port=port_num)
            r_vals.append(res)
    elif(train_type == 'batch'):
        chosen = series[seq_idxs[batch_iter % len(series)]]
        for params in solutions:
            res = exec_seqs(params, 'cma_' + train_type, [chosen], log_file=my_log, port=port_num)
            r_vals.append(res)
    elif(train_type == 'individual'):
        for params in solutions:
            res = exec_seqs(params, 'cma_' + train_type + '_' + seq, [seq], log_file=my_log, port=port_num)
            r_vals.append(res)
        
    es.tell(solutions, r_vals)
    es.logger.add()
    es.disp()
    batch_iter += 1

In [ ]:
results = {'cfg':es.best.x,'func':es.best.f,'log':my_log}
import glob
import pickle
#N=len(glob.glob('results*.pkl'))
with open('./tests/socnav/auto_{}{}_results.pkl'.format((seq if train_type == 'individual' else train_type), suffix),'wb') as fp:
    pickle.dump(results,fp)

In [ ]:
base_path = './tests/socnav/auto_{}{}_cmalogger'.format((seq if train_type == 'individual' else train_type), suffix) 

if os.path.exists(base_path):
    shutil.rmtree(base_path)
os.mkdir(base_path)    
es.logger.save_to(base_path + '/auto_{}{}_cmalogger_'.format((seq if train_type == 'individual' else train_type), suffix))

In [ ]:
# if(train_type == 'global'):
#     (res1, res2) = (0, 0)
#     res1 = exec_seqs(def_num_log, 'test_eval', series, log_file=my_log)
#     res2 = exec_seqs(es.best.x, 'test_eval', series, log_file=my_log)
# elif(train_type == 'stochastic'):
#     chosen = [random.choice(series)]
#     res1 = exec_seqs(def_num_log,'test_eval',chosen)
#     res2 = exec_seqs(es.best.x,'test_eval',chosen)

# res1,res2

In [ ]:
' '.join(['{:.03f}'.format(_) for _ in list(np.exp(es.best.x[:-1])) + [es.best.x[-1]] ])

In [ ]:
' '.join(['{:.03f}'.format(_) for _ in list(np.exp(def_num_log[:-1])) + [def_num_log[-1]] ])